In [1]:
using CSV, DataFrames, JSON, DataDrop

In [2]:
using SparseArrays
using LinearAlgebra

## Network

In [3]:
network = JSON.parsefile("../models/europe.json")

Dict{String, Any} with 11 entries:
  "bus"      => Dict{String, Any}("4304"=>Dict{String, Any}("coord"=>Any[11.843…
  "name"     => "pantagruel"
  "dcline"   => Dict{String, Any}()
  "gen"      => Dict{String, Any}("1"=>Dict{String, Any}("entsoe_names"=>Any[],…
  "branch"   => Dict{String, Any}("4304"=>Dict{String, Any}("br_r"=>0.00174284,…
  "storage"  => Dict{String, Any}()
  "switch"   => Dict{String, Any}()
  "baseMVA"  => 100
  "per_unit" => true
  "shunt"    => Dict{String, Any}()
  "load"     => Dict{String, Any}("4304"=>Dict{String, Any}("load_bus"=>4304, "…

Determine all countries in the network:

In [4]:
countries = Set([bus["country"] for bus ∈ values(network["bus"])])

Set{String} with 26 elements:
  "SI"
  "RO"
  "IT"
  "NL"
  "FR"
  "DK"
  "SK"
  "RS"
  "BG"
  "AT"
  "CH"
  "XX"
  "CZ"
  "ME"
  "DE"
  "MK"
  "BA"
  "BE"
  "AL"
  "LU"
  "HR"
  "ES"
  "GR"
  "HU"
  "PL"
  "PT"

Identify non-dispatchable generators:

In [5]:
nuc_series_df = CSV.read("../doc/data/pantagruel_nuc.csv", DataFrame);

In [6]:
nuc_ids = string.(nuc_series_df.id)

44-element Vector{String}:
 "79"
 "87"
 "160"
 "176"
 "179"
 "209"
 "210"
 "232"
 "249"
 "256"
 "257"
 "265"
 "318"
 ⋮
 "723"
 "732"
 "751"
 "757"
 "782"
 "846"
 "908"
 "917"
 "921"
 "943"
 "950"
 "982"

In [7]:
network["nuc"] = Dict{String, Any}(id => network["gen"][id] for id ∈ nuc_ids)

for id ∈ nuc_ids
    delete!(network["gen"], id)
end

In [8]:
Nlines = length(network["branch"])
Nbuses = length(network["bus"])
Nnucs = length(network["nuc"])
Ngens = length(network["gen"])

Nlines, Nbuses, Nnucs, Ngens

(8375, 4097, 44, 1039)

In [9]:
function sort_ids(data::Dict{String, Any})
    return sort(parse.(Int, keys(data)))
end

sort_ids (generic function with 1 method)

In [10]:
line_ids = sort_ids(network["branch"])
bus_ids = sort_ids(network["bus"])
gen_ids = sort_ids(network["gen"])
nuc_ids = sort_ids(network["nuc"]);

Check that there is one load at every bus:

In [11]:
sort_ids(network["load"]) == bus_ids

true

In [12]:
(line_ids == collect(1:Nlines), bus_ids == collect(1:Nbuses), gen_ids == collect(1:Ngens), 
    nuc_ids == collect(1:Nnucs))

(true, false, false, false)

**Warning**: bus IDs do not go from 1 to N

In [13]:
bus_ids_map = Dict(bus_ids[i] => i for i = 1:Nbuses);
gen_ids_map = Dict(gen_ids[i] => i for i = 1:Ngens)
nuc_ids_map = Dict(nuc_ids[i] => i for i = 1:Nnucs);

In [14]:
gen_ids_map[726]

693

Incidence matrix $M$
(encodes the connection between buses with a direction):

In [15]:
lines_from_bus = [bus_ids_map[network["branch"][string(id)]["f_bus"]] for id ∈ line_ids]
lines_to_bus = [bus_ids_map[network["branch"][string(id)]["t_bus"]] for id ∈ line_ids]

M = sparse([lines_from_bus; lines_to_bus], [1:Nlines; 1:Nlines],
    [-ones(Nlines); ones(Nlines)], Nbuses, Nlines)

4097×8375 SparseMatrixCSC{Float64, Int64} with 16750 stored entries:
⎡⣿⣶⣦⣤⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣾⣾⡷⎤
⎢⠀⠀⠑⠐⠻⠛⠿⠶⢤⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⣿⡿⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⢈⠉⣟⡟⣶⣦⣤⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢿⣿⣿⎥
⎢⠀⠀⠀⡀⠀⠀⠀⠀⡒⠒⠋⠀⢀⡁⠉⢙⢛⢲⣤⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⣿⣿⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢈⢹⢳⢶⣤⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⣿⣯⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠰⠀⠀⠀⠀⠠⠀⠄⠤⠐⠨⠾⠼⠥⣾⠿⠷⢦⣀⡀⠀⠀⠀⠀⠀⠀⠀⣻⣿⣿⎥
⎢⠀⠑⠀⡀⠀⠀⠀⠀⢄⣀⣠⡈⢂⢀⣀⣀⣀⣐⡒⣟⣓⣐⣀⣀⢰⠀⣀⣜⢉⣽⣳⣤⣄⠀⠀⠀⠀⣿⣿⣷⎥
⎢⣀⣠⣤⣤⣀⣐⣠⣀⢈⢈⣩⡀⢈⡀⠀⢈⡡⢡⣀⡍⠍⠉⠉⠉⠁⠉⠈⠉⢉⢈⣤⠀⡹⢷⣶⣄⠀⠷⠷⠞⎥
⎢⠲⡟⢟⢿⢿⣾⣟⣿⣆⣨⣭⣥⢤⣭⣤⣶⣤⡼⣼⠭⢬⣭⡩⠭⢤⡭⣭⣭⣹⠽⢾⡬⣯⣿⣿⣿⣆⠀⠀⠀⎥
⎣⠁⠛⠻⠋⠿⠻⠻⠝⠿⠽⠗⠾⠿⠾⠽⠿⠿⠯⠻⠾⠚⠛⠁⠘⠚⠛⠹⠻⠛⠛⠸⠗⠿⠾⠽⠿⠿⠆⠀⠀⎦

Susceptance matrix:

In [16]:
b = [network["branch"][string(id)]["br_b"] for id ∈ line_ids];

B = M * Diagonal(b)

4097×8375 SparseMatrixCSC{Float64, Int64} with 16750 stored entries:
⎡⣿⣶⣦⣤⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣾⣾⡷⎤
⎢⠀⠀⠑⠐⠻⠛⠿⠶⢤⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⣿⡿⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⢈⠉⣟⡟⣶⣦⣤⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢿⣿⣿⎥
⎢⠀⠀⠀⡀⠀⠀⠀⠀⡒⠒⠋⠀⢀⡁⠉⢙⢛⢲⣤⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⣿⣿⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢈⢹⢳⢶⣤⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⣿⣯⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠰⠀⠀⠀⠀⠠⠀⠄⠤⠐⠨⠾⠼⠥⣾⠿⠷⢦⣀⡀⠀⠀⠀⠀⠀⠀⠀⣻⣿⣿⎥
⎢⠀⠑⠀⡀⠀⠀⠀⠀⢄⣀⣠⡈⢂⢀⣀⣀⣀⣐⡒⣟⣓⣐⣀⣀⢰⠀⣀⣜⢉⣽⣳⣤⣄⠀⠀⠀⠀⣿⣿⣷⎥
⎢⣀⣠⣤⣤⣀⣐⣠⣀⢈⢈⣩⡀⢈⡀⠀⢈⡡⢡⣀⡍⠍⠉⠉⠉⠁⠉⠈⠉⢉⢈⣤⠀⡹⢷⣶⣄⠀⠷⠷⠞⎥
⎢⠲⡟⢟⢿⢿⣾⣟⣿⣆⣨⣭⣥⢤⣭⣤⣶⣤⡼⣼⠭⢬⣭⡩⠭⢤⡭⣭⣭⣹⠽⢾⡬⣯⣿⣿⣿⣆⠀⠀⠀⎥
⎣⠁⠛⠻⠋⠿⠻⠻⠝⠿⠽⠗⠾⠿⠾⠽⠿⠿⠯⠻⠾⠚⠛⠁⠘⠚⠛⠹⠻⠛⠛⠸⠗⠿⠾⠽⠿⠿⠆⠀⠀⎦

By construction, the transpose of the susceptance matrix $B^T$ annihilates the vector $(1, 1, \ldots, 1, 1)$:

In [17]:
B' * ones(Nbuses) == zeros(Int, Nlines)

true

Compute the pseudo-inverse of $B$:

In [18]:
Bi = pinv(Matrix(B))

8375×4097 Matrix{Float64}:
 -0.0225952     5.5164e-6     5.5164e-6    …   5.5164e-6     5.5164e-6
 -0.0225952     5.5164e-6     5.5164e-6        5.5164e-6     5.5164e-6
  1.06328e-5   -0.01088      -0.01088          1.06328e-5    1.06328e-5
  1.06328e-5   -0.01088      -0.01088          1.06328e-5    1.06328e-5
 -2.16281e-5   -2.16281e-5    0.0295151       -2.16281e-5   -2.16281e-5
  1.81255e-6    1.81255e-6   -0.00742419   …   1.81255e-6    1.81255e-6
  7.20936e-6    7.20936e-6    7.20936e-6       7.20936e-6    7.20936e-6
 -3.3705e-5    -3.39816e-5   -3.39816e-5      -1.50774e-5   -1.98764e-5
  1.473e-5      1.48213e-5    1.48213e-5       8.58272e-6    1.01665e-5
 -6.05796e-6   -6.05796e-6   -6.05796e-6      -6.05796e-6   -6.05796e-6
 -6.05796e-6   -6.05796e-6   -6.05796e-6   …  -6.05796e-6   -6.05796e-6
 -8.03995e-7   -1.04703e-6   -1.04703e-6       1.55601e-5    1.13442e-5
 -1.08941e-6   -1.08941e-6   -1.08941e-6      -1.08941e-6   -1.08941e-6
  ⋮                                    

Define $L$ as the inverse of the susceptance matrix multiplied by the susceptance:

In [19]:
L = Diagonal(b) * Bi

8375×4097 Matrix{Float64}:
 -0.499878      0.000122041   0.000122041  …   0.000122041   0.000122041
 -0.499878      0.000122041   0.000122041      0.000122041   0.000122041
  0.000488162  -0.499512     -0.499512         0.000488162   0.000488162
  0.000488162  -0.499512     -0.499512         0.000488162   0.000488162
 -0.000732243  -0.000732243   0.999268        -0.000732243  -0.000732243
  0.000244081   0.000244081  -0.999756     …   0.000244081   0.000244081
  0.000244081   0.000244081   0.000244081      0.000244081   0.000244081
 -0.000752681  -0.000758859  -0.000758859     -0.0003367    -0.000443869
  0.000996762   0.00100294    0.00100294       0.000580781   0.000687951
 -0.000244081  -0.000244081  -0.000244081     -0.000244081  -0.000244081
 -0.000244081  -0.000244081  -0.000244081  …  -0.000244081  -0.000244081
 -2.04378e-5   -2.66158e-5   -2.66158e-5       0.000395543   0.000288374
 -0.000122041  -0.000122041  -0.000122041     -0.000122041  -0.000122041
  ⋮                     

Given the power injections $P_i$ at every bus $i$ and the voltage angles difference $\Delta\theta_a$ on every line $a$, we have

$$P_i = B_{ia} \Delta \theta_a$$

The power flow of each line $a$ is computed by the susceptance times the angle difference:

$$P_a = b_a \Delta\theta_a$$

There is a thermal limit on each line giving the constraint:

$$\left| P_a \right| \leq P_a^\text{therm}$$

For the linearized version of the problem to make sense, we also require the angle differences to be less that 45°:

$$\left| \Delta\theta_a \right| \leq \frac{\pi}{4}$$

This is equivalent to a limit on the line flows, which can in some exceptional cases be more constraining than the thermal limit:

$$\left| P_a \right| \leq\frac{\pi}{4} b_a$$

We define also the maximal power flow to be:

$$P_a^\text{max} = \min\left(P_a^\text{therm}, \frac{\pi}{4} b_a \right)$$

In [20]:
line_pmax_angle = π/4 * b;

In [21]:
lines_thermal_limit = [network["branch"][string(id)]["rate_a"] for id ∈ line_ids]

count(lines_thermal_limit .> line_pmax_angle) / Nlines

0.007880597014925373

In [22]:
line_pmax = min.(lines_thermal_limit, line_pmax_angle)

8375-element Vector{Float64}:
  4.9
  4.9
  4.9
  4.9
  4.9
  4.9
  4.9
  4.9
  4.9
  4.9
  4.9
  4.9
  4.9
  ⋮
 10.0
 10.0
 10.0
 10.0
  6.902376728174339
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0

## Import generation costs

In [23]:
T = 24 * 364

8736

In [24]:
gen_costs_df = CSV.read("../doc/data/europe_noise_series.csv", DataFrame)
sort!(gen_costs_df)

Row,Column1,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,⋯
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,1,0.0600444,-0.0325225,-0.0472233,-0.0463415,-0.06765,0.116603,0.044683,-0.121029,0.0352852,-0.0475864,-0.124568,0.0007844,-0.00321964,0.0729423,0.0292253,-0.00774486,0.144995,0.0875453,0.0408881,0.0793755,0.056021,0.0983048,0.0814576,0.080336,0.124665,0.0372826,0.0254434,0.0268432,0.00380638,0.184295,0.106885,-0.0656456,0.0829781,-0.00795604,-0.0928608,0.0252056,0.015012,0.0864761,0.039864,0.00201162,0.15598,0.101852,0.0604712,0.105943,0.0909357,0.142502,0.135385,0.143921,0.197303,0.117859,0.112377,0.118154,0.0972092,0.277304,0.196934,0.0189155,0.15969,0.0588245,-0.0377113,0.0674922,0.0437345,0.101501,0.0416349,-0.00846774,0.134768,0.071867,0.0240247,0.065587,0.0493404,0.102329,0.0991598,0.11391,0.175406,0.105517,0.110502,0.12709,0.116709,0.306548,0.234565,0.0631078,0.208236,0.109242,0.0119379,0.113705,0.0839383,0.133342,0.0630896,0.000997543,0.131136,0.0545733,-0.00693216,0.0215233,-0.0067497,0.0357581,0.0240388,0.0324549,0.0900088,0.0186255,0.0245023,⋯
2,2,0.0768188,-0.00243229,-0.0465575,-0.0291031,-0.0368041,-0.051928,-0.0654904,-0.110334,-0.0768962,-0.0411255,-0.102962,-0.0823393,-0.0485936,-0.104662,-0.0834217,-0.0440392,-0.0286807,0.035831,0.00360927,-0.063598,-0.00676269,0.0259967,0.00799757,0.0384523,0.0416067,-0.0224955,-0.0528557,-0.0234163,-0.0212092,-0.0286899,-0.0369481,-0.0787882,-0.0445044,-0.00980687,-0.0743202,-0.0576069,-0.0285941,-0.0898007,-0.0736962,-0.0390762,-0.0277893,0.0335873,-0.000671154,-0.0687461,-0.0116321,0.0224396,0.00659176,0.0397737,0.0459164,-0.0152827,-0.0431815,-0.0120649,-0.00927304,-0.0175132,-0.0280542,-0.0737922,-0.0450175,-0.0173364,-0.0901718,-0.0827931,-0.0637562,-0.135147,-0.128961,-0.103502,-0.100137,-0.0449914,-0.0833991,-0.153229,-0.0952629,-0.0576312,-0.0672328,-0.0252651,-0.00806532,-0.0563174,-0.0698581,-0.0235294,-0.00528168,0.00154069,0.00503486,-0.028297,0.0107143,0.0460088,-0.02218,-0.013338,0.00390218,-0.0724769,-0.0742583,-0.0593974,-0.0687884,-0.0279844,-0.0816667,-0.167004,-0.124059,-0.100257,-0.121839,-0.0894168,-0.0788461,-0.130452,-0.143851,⋯
3,3,0.0797153,0.058979,-0.076703,0.0623828,0.0933543,0.0366714,0.0721905,0.0351522,-0.0695645,0.013812,0.0834479,-0.0662168,-0.0222337,0.000760608,-0.0827502,-0.0265702,-0.0812489,-0.137639,-0.0685121,-0.0348944,-0.075772,-0.0284962,-0.0266638,-0.133285,-0.00939197,-0.0287917,-0.160908,-0.0161586,0.0223724,-0.0251152,0.0209269,-0.00460293,-0.097188,-0.00142486,0.0804932,-0.0573325,-0.002259,0.0308135,-0.0438448,0.0198032,-0.0288936,-0.0808324,-0.00877413,0.026316,-0.0144431,0.0317395,0.0314372,-0.0781707,0.0420824,0.0185909,-0.117875,0.0224459,0.0566313,0.00501723,0.0472619,0.0183454,-0.0771616,0.0161714,0.0961536,-0.043132,0.0109213,0.0433638,-0.0315933,0.032022,-0.0165089,-0.0681503,0.00427183,0.0397314,-0.000709217,0.0456889,0.0454517,-0.0642857,0.0556024,0.0314714,-0.105946,0.0330757,0.0655794,0.0118694,0.0515755,0.0196558,-0.0793335,0.010

In [25]:
gen_costs = Matrix(gen_costs_df[gen_ids, 2:end])

1039×8736 Matrix{Float64}:
  0.0600444   -0.0325225   -0.0472233  …   0.0335081     0.0230842
  0.0768188   -0.00243229  -0.0465575      0.0756779     0.0897179
  0.0797153    0.058979    -0.076703       0.058288     -0.0451257
 -0.0365876   -0.00437924  -0.111027      -0.000830024   0.00731655
 -0.0648586   -0.0997798   -0.0615101     -0.06673       0.0585788
  0.105941     0.0743317    0.127231   …   0.12184       0.0610914
 -0.162458    -0.0857404   -0.0771757     -0.059919     -0.103284
 -0.0377599   -0.0153825    0.0201534      0.0741944     0.0488174
 -0.0659082    0.0281031    0.0134927      0.0712088     0.0515408
  0.127192     0.112071     0.0449865     -0.0156324    -0.00585513
 -0.0108403    0.0633816   -0.016528   …  -0.00327896    0.00268543
  0.153984     0.133841     0.162632       0.251535      0.228895
 -0.226887    -0.0866635   -0.0261829     -0.0491846    -0.0730636
  ⋮                                    ⋱                 ⋮
 -0.00188743  -0.135833    -0.0828897     

Distribution of generators on buses:

In [26]:
gen_buses = [bus_ids_map[network["gen"][string(id)]["gen_bus"]] for id ∈ gen_ids]

A_gen = sparse(gen_buses, 1:Ngens, ones(Ngens), Nbuses, Ngens)

4097×1039 SparseMatrixCSC{Float64, Int64} with 1039 stored entries:
⎡⢳⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⡅⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠹⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⢳⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠸⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⢹⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠈⣇⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠸⡄⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠙⣆⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠘⣆⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠘⡄⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⢇⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⢸⡀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⢧⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠸⡄⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢹⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⡇⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢹⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠸⡄⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡇⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡇⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡇⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠱⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠸⎦

Maximum capacity for each generator:

In [27]:
gen_pmax = [convert(Float64, network["gen"][string(id)]["pmax"]) for id ∈ gen_ids]

1039-element Vector{Float64}:
  1.14
  1.6
  0.88
  0.6
  0.27
  0.23
  0.24
  8.45
  3.0
  5.5
  3.0
  3.33600006
  2.1
  ⋮
  0.114
  0.16699999999999998
  0.12
  0.14
 15.8539
  1.05
  0.183
  0.19699999999999998
  0.18600000000000003
  0.256
  0.2475
  0.384

Expected production for each generator:

In [28]:
gen_pexp = [convert(Float64, network["gen"][string(id)]["pexp"]) for id ∈ gen_ids]

1039-element Vector{Float64}:
 0.18403633308432765
 0.30949227057286455
 0.36802561812197143
 0.25092655781043505
 0.05904927085954283
 0.030972414892266358
 0.03231904162671272
 2.516608703642022
 1.325864324040231
 2.4307512607404234
 0.893470545671724
 0.4492346866910672
 0.45927210668533314
 ⋮
 0.011307311244368606
 0.03534936879747013
 0.02540074404608632
 0.024567999376244737
 2.819572725820961
 0.5005519988019325
 0.032113884898948473
 0.03457068483657294
 0.032640342028439434
 0.04351682579693602
 0.049584314579468325
 0.07693081534753873

## Import nuclear profiles

In [29]:
nuclear_profiles = Matrix(nuc_series_df[:, 2:end]) / 100.

44×8736 Matrix{Float64}:
 19.7042   19.7052   19.7062   19.7062   …  19.7042   19.7052   19.7052
 10.624    10.623    10.629    10.629       10.633    10.63     10.624
  9.75114   9.75404   9.75791   9.75888     10.4758    9.80338   9.75114
  9.294     9.279     9.291     9.293        9.299     9.296     9.296
 19.1431   19.1431   19.1431   19.1431      19.1431   19.1431   19.1431
 12.7429   12.736    12.477    12.7685   …  12.7764   12.7321   12.7311
 12.0824   12.0824   12.0787   12.0787      12.1026   12.1024   12.0844
 13.64     13.6173   13.553    13.53        13.6965   13.6178   13.635
 12.7271   12.7467   12.7437   12.7301      12.9856   12.9398   12.9096
 12.0348   12.0338   12.0309   12.023       12.0309   12.0328   12.0348
 12.8447   12.8466   12.8437   12.8437   …  12.8234   12.8251   12.8417
 14.0004   13.997    13.8992   13.7054      14.0317   14.0294   14.0088
 17.21     17.21     17.21     17.21        17.26     17.26     17.21
  ⋮                                      ⋱ 

In [30]:
nuc_buses = [bus_ids_map[network["nuc"][string(id)]["gen_bus"]] for id ∈ nuc_ids]

A_nuc = sparse(nuc_buses, 1:Nnucs, ones(Nnucs), Nbuses, Nnucs)

4097×44 SparseMatrixCSC{Float64, Int64} with 44 stored entries:
⎡⠀⠀⎤
⎢⠃⠀⎥
⎢⠀⠀⎥
⎢⠁⠀⎥
⎢⠁⠀⎥
⎢⠆⠀⎥
⎢⠸⠀⎥
⎢⢀⠀⎥
⎢⢀⠀⎥
⎢⠰⠀⎥
⎢⠠⠀⎥
⎢⠈⠆⎥
⎢⠀⡂⎥
⎢⠀⡆⎥
⎢⠀⠆⎥
⎢⠀⠐⎥
⎢⠀⠠⎥
⎢⠀⠰⎥
⎢⠀⠐⎥
⎢⠀⠈⎥
⎢⠀⠀⎥
⎢⠀⠀⎥
⎢⠀⠀⎥
⎢⠀⠀⎥
⎢⠀⠀⎥
⎣⠀⠀⎦

In [31]:
avg_nuclear = sum(nuclear_profiles, dims=2) / T

44×1 Matrix{Float64}:
 18.07435324434912
 10.52281204212462
 16.740458566186987
 12.772566391941368
 12.115955164116976
  9.503013685447213
 12.653122996045495
 11.144890212912095
 11.818796142967246
 10.666854506547292
 10.062367679398305
 12.784686302117901
  9.543592032967025
  ⋮
 35.7308990489626
 36.51960794413923
 26.428929067784257
 18.913049255210407
 16.863302806091106
  4.221288303765866
  6.948160097272817
  3.3505223372592856
  9.220073416218261
  3.427495421245371
 12.033299969955593
 22.566314102564032

In [32]:
total_nuclear = sum(nuclear_profiles, dims=1)[1, :]

8736-element Vector{Float64}:
 881.9648751409237
 881.7314571887553
 881.2714462952173
 881.7447030905244
 881.6503497966312
 884.6994189318873
 893.6933152593614
 894.6164691496289
 895.1912581211678
 895.7077659455551
 896.8124050182465
 896.0096340851055
 896.5861009733926
   ⋮
 888.7747270743923
 888.9280001324361
 889.4097888424648
 888.7765443288088
 887.5665997127107
 886.6996310914843
 884.5294240239839
 884.3841469805394
 882.2464379503751
 882.5014211344914
 881.1820477773415
 881.1772241894436

## Calculate total load by country based on capacity and usage

Determine the total expected production:

In [33]:
expected_production_by_country = Dict(country => 0.0 for country ∈ countries)

for gen ∈ values(network["gen"])
    expected_production_by_country[gen["country"]] += gen["pexp"]
end

sort(expected_production_by_country, byvalue=true, rev=true)

OrderedCollections.OrderedDict{String, Float64} with 26 entries:
  "DE" => 279.256
  "PL" => 136.378
  "FR" => 129.988
  "IT" => 127.814
  "ES" => 78.738
  "NL" => 59.6867
  "AT" => 57.8544
  "RO" => 48.9335
  "GR" => 48.2076
  "PT" => 42.5945
  "CZ" => 35.501
  "RS" => 29.6354
  "BG" => 19.8427
  "BE" => 17.3629
  "BA" => 13.4466
  "CH" => 11.6612
  "SI" => 11.1836
  "HU" => 10.6986
  "SK" => 9.39437
  "DK" => 7.71758
  "HR" => 6.7509
  "MK" => 5.00832
  "AL" => 4.75198
  "LU" => 3.56473
  "XX" => 2.6395
  "ME" => 2.33425

Export by country:

In [34]:
out_flow_df = CSV.read("pantagruel_out_flow.csv", DataFrame)

Row,country,out_flow
,String3,Float64
1,DE,5663.42
2,SI,127.709
3,HR,-717.664
4,BE,-725.658
5,ES,-1454.95
6,FR,3016.91
7,RS,167.457
8,DK,-282.057
9,BA,425.84


In [35]:
out_flow_by_country = Dict{String, Float64}()
for (country, flow) in eachrow(out_flow_df)
    out_flow_by_country[country] = flow / 100.0
end
out_flow_by_country["XX"] = 0.0

sort(out_flow_by_country, byvalue=true, rev=true)

OrderedCollections.OrderedDict{String, Float64} with 26 entries:
  "DE" => 56.6342
  "FR" => 30.1691
  "CZ" => 12.5113
  "RO" => 6.64282
  "PT" => 5.73057
  "BA" => 4.2584
  "PL" => 2.34598
  "BG" => 1.89876
  "RS" => 1.67457
  "SI" => 1.27709
  "AL" => 0.0488879
  "LU" => 0.0
  "XX" => 0.0
  "ME" => -0.309394
  "MK" => -1.36804
  "DK" => -2.82057
  "CH" => -5.13167
  "SK" => -5.37451
  "HR" => -7.17664
  "BE" => -7.25658
  "GR" => -8.38198
  "HU" => -9.63781
  "NL" => -9.9127
  "AT" => -9.93982
  "ES" => -14.5495
  "IT" => -41.3325

Check: total out flow adds up to zero

In [36]:
sum(values(out_flow_by_country))

1.865174681370263e-14

Nuclear production by country:

In [37]:
nuclear_production_by_country = Dict(country => 0.0 for country ∈ countries)
for id ∈ nuc_ids
    country = network["nuc"][string(id)]["country"]
    nuclear_production_by_country[country] += avg_nuclear[nuc_ids_map[id]]
end
nuclear_production_by_country

Dict{String, Float64} with 26 entries:
  "SI" => 0.0
  "ES" => 45.3376
  "GR" => 0.0
  "LU" => 0.0
  "HU" => 16.8633
  "CH" => 22.9463
  "MK" => 0.0
  "RS" => 0.0
  "XX" => 0.0
  "DE" => 90.667
  "AL" => 0.0
  "FR" => 441.027
  "PL" => 0.0
  "IT" => 0.0
  "AT" => 0.0
  "HR" => 0.0
  "PT" => 0.0
  "ME" => 0.0
  "NL" => 4.22129
  "BE" => 46.9143
  "RO" => 11.829
  "BG" => 17.1781
  "CZ" => 24.8885
  "SK" => 16.7945
  "DK" => 0.0
  "BA" => 0.0

Determine annual load by country:

In [38]:
expected_load_by_country = Dict(country => prod + nuclear_production_by_country[country] - out_flow_by_country[country]
    for (country, prod) ∈ expected_production_by_country)

sort(expected_load_by_country, byvalue=true, rev=true)

OrderedCollections.OrderedDict{String, Float64} with 26 entries:
  "FR" => 540.846
  "DE" => 313.289
  "IT" => 169.147
  "ES" => 138.625
  "PL" => 134.032
  "NL" => 73.8207
  "BE" => 71.5338
  "AT" => 67.7943
  "GR" => 56.5896
  "RO" => 54.1198
  "CZ" => 47.8782
  "CH" => 39.7391
  "HU" => 37.1997
  "PT" => 36.8639
  "BG" => 35.122
  "SK" => 31.5634
  "RS" => 27.9608
  "HR" => 13.9275
  "DK" => 10.5382
  "SI" => 9.90647
  "BA" => 9.18824
  "MK" => 6.37635
  "AL" => 4.70309
  "LU" => 3.56473
  "ME" => 2.64364
  "XX" => 2.6395

In [39]:
total_expected_load = sum(values(expected_load_by_country))

1939.6111574858896

## Import loads

In [40]:
normalized_loads_df = CSV.read("../doc/data/europe_load_series.csv", DataFrame)
sort!(normalized_loads_df)

Row,Column1,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,⋯
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,81,1.13557,1.12543,1.11811,1.14711,1.25385,1.37139,1.446,1.51316,1.53222,1.52428,1.50463,1.48967,1.4727,1.48166,1.52101,1.60451,1.67275,1.66746,1.59331,1.53063,1.49717,1.39425,1.27845,1.2032,1.16054,1.14112,1.15439,1.18785,1.2978,1.41831,1.50578,1.55338,1.55392,1.53218,1.48875,1.46694,1.45669,1.47129,1.48591,1.57232,1.63949,1.63516,1.58809,1.51374,1.46829,1.37922,1.26011,1.17412,1.12646,1.11057,1.11425,1.14151,1.25516,1.36759,1.45238,1.52336,1.52098,1.52801,1.51139,1.51271,1.51358,1.52449,1.53442,1.58182,1.61785,1.59167,1.52642,1.45826,1.43254,1.33536,1.21968,1.15093,1.10673,1.08018,1.07372,1.10321,1.19584,1.32098,1.41412,1.48685,1.47159,1.47905,1.46417,1.45375,1.44228,1.46285,1.47501,1.54374,1.5998,1.56841,1.50165,1.44789,1.40852,1.3283,1.2189,1.1405,1.09921,1.07686,1.07496,⋯
2,82,0.933537,0.910476,0.925002,0.960108,1.05485,1.16503,1.25982,1.31265,1.29448,1.27865,1.23777,1.21287,1.20373,1.21998,1.26972,1.36712,1.44082,1.44791,1.3882,1.33126,1.29705,1.20425,1.1016,1.02598,0.98655,0.974642,0.983692,1.0203,1.13066,1.2554,1.35606,1.38741,1.37277,1.35574,1.33204,1.31411,1.30946,1.32905,1.36069,1.44333,1.50083,1.49369,1.43523,1.37245,1.34329,1.25786,1.15988,1.0877,1.06239,1.0449,1.05472,1.08266,1.18771,1.31772,1.40735,1.45823,1.44996,1.4245,1.39026,1.36981,1.34431,1.3662,1.4152,1.51651,1.57432,1.55636,1.48764,1.43414,1.39571,1.31621,1.2167,1.15145,1.11369,1.0961,1.10778,1.13385,1.22726,1.35268,1.43855,1.48632,1.47354,1.45707,1.42515,1.42968,1.39859,1.41449,1.4435,1.52882,1.60496,1.59519,1.53484,1.47082,1.43776,1.35384,1.21952,1.15665,1.11242,1.09566,1.0965,⋯
3,83,0.867103,0.871388,0.887594,0.934847,1.03289,1.15492,1.29417,1.38025,1.37758,1.3446,1.30762,1.29106,1.27524,1.27567,1.32708,1.40379,1.44205,1.43922,1.37918,1.31347,1.24057,1.13432,1.03283,0.973265,0.946528,0.936659,0.955834,1.00069,1.13585,1.27458,1.36207,1.37961,1.31653,1.26312,1.20615,1.17798,1.162,1.17141,1.20031,1.30206,1.3866,1.38555,1.35604,1.28748,1.2248,1.11789,1.06297,1.00967,0.983012,0.973727,0.985597,1.0181,1.13939,1.25962,1.34336,1.36614,1.31082,1.28424,1.23577,1.21744,1.2113,1.22474,1.26525,1.33397,1.38184,1.35734,1.30359,1.23742,1.2044,1.11634,1.0321,0.975785,0.944397,0.926087,0.938364,0.976097,1.06689,1.18603,1.26029,1.27651,1.21103,1.16559,1.12184,1.11027,1.0964,1.10657,1.13955,1.24638,1.32262,1.28996,1.24824,1.1925,1.16881,1.11008,0.966294,0.921536,0.894437,0.875372,0.885203,⋯
4,84,0.798439,0.785086,0.785474,0.798889,0.8276,0.905052,0.999134,1.09612,1.1229,1.11367,1.0836,1.0563,1.03366,1.03628,1.08386,1.18209,1.23162,1.21938,1.16374,1.09299,1.05526,0.984559,0.908005,0.844014,0.817389,0.80362,0.80814,0.835429,0.904441,1.01226,1.13212,1.20414,1.20532,1.18233,1.15835,1.14131,1.11586,1.12052,1.1399,1.21967,1.26368,1.24594,1.19577,1.13278,1.10189,1.03091,0.94543,0.882875,0.852449,0.829925,0.834052,0.856704,0.929421,1.02438,1.15197,1.23932,1.23458,1.21177,1.1664,1.12684,1.0867,1.0

Check that the IDs match:

In [41]:
normalized_loads_df[:, 1] == bus_ids

true

In [42]:
normalized_loads = Matrix(normalized_loads_df[:, 2:end])

4097×8736 Matrix{Float64}:
 1.13557   1.12543   1.11811   1.14711   …  1.35167   1.24511   1.17542
 0.933537  0.910476  0.925002  0.960108     1.11518   1.01182   0.962993
 0.867103  0.871388  0.887594  0.934847     0.982226  0.917873  0.869044
 0.798439  0.785086  0.785474  0.798889     0.972084  0.886739  0.833901
 0.8401    0.805474  0.793918  0.782738     1.04626   0.956483  0.90332
 0.733483  0.710304  0.695572  0.698443  …  0.899803  0.836091  0.789785
 0.92908   0.919089  0.912357  0.937145     1.10438   1.02501   0.967391
 0.955082  0.958165  0.985505  1.04787      1.06848   0.994147  0.9486
 1.05564   1.04208   1.03689   1.0604       1.26612   1.1612    1.09677
 1.03456   1.01863   1.03112   1.07418      1.21535   1.11583   1.06174
 0.951279  0.91725   0.912524  0.924708  …  1.17415   1.07007   1.01327
 1.16986   1.15464   1.14998   1.1736       1.38071   1.27374   1.20869
 0.962154  0.94282   0.94471   0.963393     1.16267   1.06001   1.00197
 ⋮                               

Check normalization:

In [43]:
sum(normalized_loads) / T / Nbuses

1.0001457896701273

Split into countries:

In [44]:
bus_country = Dict(bus["index"] => bus["country"] for bus ∈ values(network["bus"]))

normalized_annual_load = sum(normalized_loads, dims=2) / T
normalized_annual_load_by_country = Dict(country => 0.0 for country in countries)
for id ∈ bus_ids
    normalized_annual_load_by_country[bus_country[id]] += normalized_annual_load[bus_ids_map[id]]
end

normalized_annual_load_by_country

Dict{String, Float64} with 26 entries:
  "SI" => 15.0
  "ES" => 908.0
  "GR" => 38.0
  "LU" => 12.0468
  "HU" => 46.9921
  "CH" => 163.0
  "MK" => 12.0
  "RS" => 53.0
  "XX" => 13.0
  "DE" => 560.385
  "AL" => 30.0
  "FR" => 865.0
  "PL" => 188.0
  "IT" => 324.0
  "AT" => 78.1372
  "HR" => 28.0
  "PT" => 146.0
  "ME" => 10.0
  "NL" => 38.0081
  "BE" => 50.028
  "RO" => 124.0
  "BG" => 64.0
  "CZ" => 71.0
  "SK" => 43.0
  "DK" => 182.0
  "BA" => 35.0

Check that the load assignement within each country is properly normalized:

In [45]:
load_prop_by_country = Dict(country => 0.0 for country in countries)
for bus ∈ values(network["bus"])
    load_prop_by_country[bus["country"]] += bus["load_prop"]
end

load_prop_by_country

Dict{String, Float64} with 26 entries:
  "SI" => 1.0
  "ES" => 1.0
  "GR" => 1.0
  "LU" => 1.0
  "HU" => 1.0
  "CH" => 1.0
  "MK" => 1.0
  "RS" => 1.0
  "XX" => 1.0
  "DE" => 1.0
  "AL" => 1.0
  "FR" => 1.0
  "PL" => 1.0
  "IT" => 1.0
  "AT" => 1.0
  "HR" => 1.0
  "PT" => 1.0
  "ME" => 1.0
  "NL" => 1.0
  "BE" => 1.0
  "RO" => 1.0
  "BG" => 1.0
  "CZ" => 1.0
  "SK" => 1.0
  "DK" => 1.0
  "BA" => 1.0

Compute a scaling value for each country:

In [46]:
load_scaling_by_country = Dict(country => expected_load_by_country[country] / value
    for (country, value) ∈ normalized_annual_load_by_country)

Dict{String, Float64} with 26 entries:
  "SI" => 0.660432
  "ES" => 0.152671
  "GR" => 1.4892
  "LU" => 0.295907
  "HU" => 0.791616
  "CH" => 0.243798
  "MK" => 0.531363
  "RS" => 0.527562
  "XX" => 0.203038
  "DE" => 0.55906
  "AL" => 0.15677
  "FR" => 0.625255
  "PL" => 0.712935
  "IT" => 0.522057
  "AT" => 0.867631
  "HR" => 0.497412
  "PT" => 0.252493
  "ME" => 0.264364
  "NL" => 1.94224
  "BE" => 1.42988
  "RO" => 0.43645
  "BG" => 0.548781
  "CZ" => 0.674341
  "SK" => 0.734032
  "DK" => 0.0579019
  "BA" => 0.262521

In [47]:
load_scaling = [(
        network["bus"][string(id)]["load_prop"]
        * normalized_annual_load_by_country[network["bus"][string(id)]["country"]]
        * load_scaling_by_country[network["bus"][string(id)]["country"]]
        )
    for id ∈ bus_ids]

loads = load_scaling .* normalized_loads

4097×8736 Matrix{Float64}:
 0.721026   0.714589   0.709939   …  0.858237   0.790578   0.746327
 0.50816    0.495607   0.503514      0.607034   0.550774   0.524195
 0.30487    0.306377   0.312075      0.345347   0.322721   0.305552
 0.216517   0.212897   0.213002      0.263606   0.240462   0.226134
 0.144991   0.139015   0.13702       0.180571   0.165077   0.155901
 0.232407   0.225062   0.220394   …  0.285106   0.264918   0.250246
 0.456641   0.451731   0.448422      0.5428     0.503792   0.475471
 0.203504   0.204161   0.209987      0.227667   0.211828   0.202123
 0.449859   0.444083   0.44187       0.539556   0.494846   0.467391
 0.239064   0.235385   0.23827       0.280842   0.257846   0.245347
 0.195259   0.188275   0.187304   …  0.241006   0.219643   0.207983
 0.153156   0.151163   0.150553      0.180759   0.166756   0.158239
 0.205415   0.201287   0.20169       0.248224   0.226307   0.213915
 ⋮                                ⋱                        ⋮
 0.25982    0.253145   0.246

Check that the annual load matches the expectations:

In [48]:
annual_load = sum(loads, dims=2) / T
annual_load_by_country = Dict(country => 0.0 for country in countries)
for id ∈ bus_ids
    annual_load_by_country[bus_country[id]] += annual_load[bus_ids_map[id]]
end

Dict(country => annual_load_by_country[country] / expected_load_by_country[country] for country ∈ countries)

Dict{String, Float64} with 26 entries:
  "SI" => 1.0
  "ES" => 1.0
  "GR" => 1.0
  "LU" => 1.00393
  "HU" => 0.999829
  "CH" => 1.0
  "MK" => 1.0
  "RS" => 1.0
  "XX" => 1.0
  "DE" => 1.00069
  "AL" => 1.0
  "FR" => 1.0
  "PL" => 1.0
  "IT" => 1.0
  "AT" => 1.00177
  "HR" => 1.0
  "PT" => 1.0
  "ME" => 1.0
  "NL" => 1.00019
  "BE" => 1.00053
  "RO" => 1.0
  "BG" => 1.0
  "CZ" => 1.0
  "SK" => 1.0
  "DK" => 1.0
  "BA" => 1.0

In [49]:
total_load = sum(loads, dims=1)[1, :]

8736-element Vector{Float64}:
 1717.8513403491056
 1672.0906231760202
 1630.2834745123464
 1631.298945846584
 1707.2237080121654
 1870.3235084671355
 2048.3796169769334
 2170.4252174910994
 2242.534682999395
 2283.1801618714762
 2308.655024587559
 2320.8099868509807
 2297.9049453808148
    ⋮
 1958.5235665981604
 1914.4888277748246
 1895.8123829925985
 1929.9392953057663
 2036.619395485746
 2115.3202400064374
 2128.155557160382
 2077.049343041922
 1999.7682241667403
 1958.1652469758737
 1890.3303618160303
 1792.8398209578002

## Ramp constraints

In [50]:
ramp_gen_ids = sort([parse(Int, id) for (id, gen) ∈ network["gen"] if gen["aggregated_type"] == "coal"])

167-element Vector{Int64}:
    9
   10
   23
   35
   36
   37
   39
   42
   68
   72
  100
  126
  180
    ⋮
  947
  948
  969
  973
  975
  981
 1004
 1005
 1018
 1019
 1061
 1077

In [51]:
N_ramp_constraints = length(ramp_gen_ids)

167

In [52]:
A_gen_ramp = sparse(1:N_ramp_constraints, [gen_ids_map[id] for id ∈ ramp_gen_ids], ones(N_ramp_constraints), N_ramp_constraints, Ngens)

167×1039 SparseMatrixCSC{Float64, Int64} with 167 stored entries:
⎡⠈⠉⠐⠐⠐⠀⠐⠦⢄⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠉⠙⠲⢤⣀⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠁⠀⠉⠉⠁⠀⠀⠀⠈⠀⠀⠀⠒⠒⠒⠲⠤⠀⠤⢤⢀⡀⢀⢀⎦

In [53]:
gen_ramp = 2. * ones(N_ramp_constraints);

## Cost computation

Quadratic cost for each line:

In [54]:
line_costs = 1. ./ lines_thermal_limit;

Quadratic term:

In [55]:
LA = L * A_gen

quadratic_cost = LA' * (line_costs .* LA)

1039×1039 Matrix{Float64}:
  0.586155     0.201792     0.201592    …  -0.057275    0.0602631   0.0602631
  0.201792     0.69641      0.696211       -0.0580143   0.0592317   0.0592317
  0.201592     0.696211     1.10418        -0.0582136   0.0590324   0.0590324
  0.201592     0.696211     0.900093       -0.0582136   0.0590324   0.0590324
  0.128966     0.127872     0.127673       -0.0553122   0.0561581   0.0561581
  0.131182     0.129961     0.129762    …  -0.0551642   0.0564105   0.0564105
  0.127573     0.12675      0.126551       -0.0553062   0.0563286   0.0563286
  0.15602      0.152999     0.152799       -0.0566643   0.0573773   0.0573773
  0.15602      0.152999     0.152799       -0.0566643   0.0573773   0.0573773
  0.15602      0.152999     0.152799       -0.0566643   0.0573773   0.0573773
  0.156096     0.153071     0.152872    …  -0.0566116   0.0574311   0.0574311
  0.155886     0.15285      0.152651       -0.0564959   0.0575183   0.0575183
  0.155886     0.15285      0.152651 

Linear term:

In [71]:
linear_cost = 2 * LA' * (line_costs .* L * (A_nuc * nuclear_profiles - loads))

1039×8736 Matrix{Float64}:
 -50.8009  -50.2454  -50.8867  -52.6162  …  -57.9082  -52.9049  -50.5614
 -51.9204  -51.3979  -52.1173  -53.9156     -59.0852  -53.9697  -51.656
 -51.9996  -51.4948  -52.2356  -54.0516     -59.1334  -54.0101  -51.7111
 -51.9636  -51.4567  -52.1951  -54.0059     -59.1     -53.9765  -51.6787
 -46.8604  -46.3034  -46.8339  -48.373      -53.513   -48.8708  -46.6336
 -47.1264  -46.5489  -47.0662  -48.6096  …  -53.8578  -49.1872  -46.907
 -46.1676  -45.629   -46.1726  -47.7034     -52.6705  -48.0957  -45.9189
 -55.1316  -54.3282  -54.7666  -56.4796     -63.3756  -57.8673  -54.9134
 -55.1316  -54.3282  -54.7666  -56.4796     -63.3756  -57.8673  -54.9134
 -55.1316  -54.3282  -54.7666  -56.4796     -63.3756  -57.8673  -54.9134
 -54.5091  -53.7298  -54.1848  -55.8912  …  -62.604   -57.1797  -54.2885
 -53.4924  -52.7476  -53.219   -54.9083     -61.3705  -56.0597  -53.2661
 -53.4924  -52.7476  -53.219   -54.9083     -61.3705  -56.0597  -53.2661
   ⋮                      

## Export the data

In [57]:
DataDrop.store_matrix("PTDF_data/P_max_gen.h5", gen_pmax)

In [58]:
DataDrop.store_matrix("PTDF_data/P_exp_gen.h5", gen_pexp)

In [59]:
DataDrop.store_matrix("PTDF_data/P_total.h5", total_load - total_nuclear)

In [60]:
DataDrop.store_matrix("PTDF_data/A_gen_ramp.h5", A_gen_ramp)

In [61]:
DataDrop.store_matrix("PTDF_data/gen_ramp.h5", gen_ramp)

In [73]:
DataDrop.store_matrix("PTDF_data/linear_line_cost.h5", linear_cost)

In [63]:
DataDrop.store_matrix("PTDF_data/linear_gen_cost.h5", gen_costs)

In [64]:
DataDrop.store_matrix("PTDF_data/quadratic_cost.h5", quadratic_cost)

Data used for analysis of results:

In [65]:
DataDrop.store_matrix("PTDF_data/A_gen.h5", A_gen)

In [74]:
DataDrop.store_matrix("PTDF_data/P_load.h5", loads)

In [67]:
DataDrop.store_matrix("PTDF_data/A_nuc.h5", A_nuc)
DataDrop.store_matrix("PTDF_data/P_nuc.h5", nuclear_profiles)

In [68]:
DataDrop.store_matrix("PTDF_data/PTDF_matrix.h5", L)

In [69]:
DataDrop.store_matrix("PTDF_data/thermal_limits.h5", convert(Vector{Float64}, lines_thermal_limit))

In [70]:
DataDrop.store_matrix("PTDF_data/susceptance.h5", b)